In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np

In [3]:
df3 = pd.read_csv("../data/cleaned_data.csv")
len(df3)
print(df3.head())
print(df3.dtypes)

                                           job_title  \
0  Performance Analyst ( Material) – Analyste per...   
1                           Analyst, Capital Markets   
2                              Laboratory Technician   
3                                  Insurance Analyst   
4            Part Time Sales Associate- CALVIN KLEIN   

                                          job_skills  
0  SAP, DRMIS, Data warehousing, Data analysis, D...  
1  Debt and equity offering memorandums, Financia...  
2  Laboratory Technician, Pharmaceutical Testing,...  
3  Excel, Risk Management, Property Management, I...  
4  Customer service, Suggestive selling, UPT, Con...  
job_title     object
job_skills    object
dtype: object


In [4]:
df3 = df3.dropna(subset=["job_title", "job_skills"]).reset_index(drop=True)

In [5]:
import sys
import os

sys.path.append(os.path.abspath(".."))
import constants
tech_title_anchors=list(set(constants.tech_title_anchors))
constants.tech_title_anchors

Loaded keys: 12508
{'udeploy': 'devops and deployment', 'av/vc': 'video conferencing', 'thinks holistically': 'soft skills', 'checkpoint firewall management': 'checkpoint firewall', 'management plans': 'project management', 'sops': 'standard operating procedures', 'microsoft graph': 'microsoft apis', 'software asset management': 'software development', 'business terminology': 'business unit', 'data sets': 'datasets', 'retail operations': 'retail', 'step definitions': 'software testing', 'elisa/msd': 'immunoassays', 'sae j3061': 'automotive standards and security', 'react with redux': 'web development', 'platform as a service': 'paas', 'source control': 'version control', 'esra': 'risk management', 'disaster recovery solutions': 'disaster recovery', 'reporting projects': 'reporting', 'industrial automation engineering tools': 'industrial automation', 'medicine': 'healthcare', 'i/o technologies': 'input/output systems', 'sales process management': 'sales', 'r (programming language)': 'r'

['software engineer',
 'software developer',
 'backend engineer',
 'frontend engineer',
 'full-stack engineer',
 'systems engineer',
 'platform engineer',
 'site reliability engineer',
 'devops engineer',
 'cloud engineer',
 'infrastructure engineer',
 'application developer',
 'embedded software engineer',
 'mobile engineer',
 'data scientist',
 'data analyst',
 'machine learning engineer',
 'ai engineer',
 'applied scientist',
 'research scientist',
 'data engineer',
 'business intelligence analyst',
 'ml research engineer',
 'deep learning engineer',
 'nlp engineer',
 'computer vision engineer',
 'mlops engineer',
 'cybersecurity engineer',
 'security analyst',
 'security engineer',
 'information security engineer',
 'network engineer',
 'it support engineer',
 'systems administrator',
 'product manager',
 'technical program manager',
 'business analyst',
 'quantitative analyst',
 'data product manager',
 'operations analyst',
 'robotics engineer',
 'ar/vr engineer',
 'autonomous sy

In [6]:
pwd

'/Users/anshikabajpai/Desktop/data_mining/CareerPathGuidanceTool/notebooks'

In [7]:
# 2. Pre-lowercase title once
df3["job_title_lower"] = df3["job_title"].str.lower()

# 3. Filter ONLY on title (ignore skills)
def is_tech_row(row):
    title = row["job_title_lower"]
    return any(w in title for w in tech_title_anchors)

df3["is_tech"] = df3.apply(is_tech_row, axis=1)
df_tech = df3[df3["is_tech"]].reset_index(drop=True)

print("Total rows:", len(df3))
print("Tech rows (title-based):", len(df_tech))
df_tech[["job_title", "job_skills"]].head()


Total rows: 110605
Tech rows (title-based): 2182


,job_title,job_skills
0,RF Systems Engineer,"RF Systems Engineering, COFDM Modulation Schem..."
1,Security Engineer Associate,"Security Engineering, Physical Security, Camer..."
2,Cyber Security Analyst,"Security and Risk Management, Asset Security, ..."
3,"Business Analyst 4 - Lansing, MI - 12+ Months ...","SQL Server, Power BI, Business Requirements Ga..."
4,Systems Administrator,"Windows computing, Server 2012, Software troub..."


## Binarization of Tech

In [8]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter
from itertools import chain

import sys
import os

sys.path.append(os.path.abspath(".."))
from constants import  BASIC_SKILLS, EDUCATION_SKILLS, EXTENDED_MAPPING, NON_RELEVANT_SKILLS_MAPPING, BASIC_AND_ADDITIONAL_SKILLS

In [9]:
import sys
import os

sys.path.append(os.path.abspath(".."))
from src.utils import split_normalize_and_canonicalize



In [10]:
print(df_tech)
# Create your skills_list variable
df_tech["skills_list"] = df_tech["job_skills"].apply(split_normalize_and_canonicalize)

                                              job_title  \
0                                   RF Systems Engineer   
1                           Security Engineer Associate   
2                                Cyber Security Analyst   
3     Business Analyst 4 - Lansing, MI - 12+ Months ...   
4                                 Systems Administrator   
...                                                 ...   
2177  Iris Systems Integration Business Analyst (Rem...   
2178                                 Sales Data Analyst   
2179                           Optical Network Engineer   
2180                       Information Security Analyst   
2181                         Data Engineering Architect   

                                             job_skills  \
0     RF Systems Engineering, COFDM Modulation Schem...   
1     Security Engineering, Physical Security, Camer...   
2     Security and Risk Management, Asset Security, ...   
3     SQL Server, Power BI, Business Requirements Ga...

In [11]:
skills_list=df_tech["skills_list"]
skills_list

0       [.net, asset management, broadcasting, cofdm m...
1       [attention to detail, camera management, colla...
2       [.net, business unit, ccnpsecurity, communicat...
3       [.net, auditing, azure devops, business requir...
4       [analysis, azure ad, business analysis, custom...
                              ...                        
2177    [analytical thinking, business analysis, busin...
2178    [accounting, analytical thinking, business ana...
2179    [design, network engineering, optical transpor...
2180    [communication, compliance, data privacy regul...
2181    [aws, aws emr, data analysis, data architect, ...
Name: skills_list, Length: 2182, dtype: object

In [12]:
mlb = MultiLabelBinarizer(sparse_output=True)  # keeps all skills, one column each
X_sparse = mlb.fit_transform(skills_list)

print("Number of unique skills (columns):", len(mlb.classes_))
print("Matrix shape:", X_sparse.shape)  # (n_jobs, n_unique_skills)

Number of unique skills (columns): 6397
Matrix shape: (2182, 6397)


In [13]:
skills_df = pd.DataFrame.sparse.from_spmatrix(
    X_sparse,
    index=df_tech.index,
    columns=mlb.classes_
)

# Final dataset: job_title + all skill columns
df_tech_bin = pd.concat(
    [df_tech[["job_title"]].reset_index(drop=True),
     skills_df.reset_index(drop=True)],
    axis=1
)

df_tech_bin.head()


,job_title,(rt)qpcr analysis,* energy efficiency,* fire / life safety systems,* graphics manipulation programs,* semantic web,* telegraf,.net,.net development,.net maui,...,x86 architecture,xaml,xg boost,xilinx mpsoc,xml,xp,xray imaging,yaml,zabbix,zero trust
0,RF Systems Engineer,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Security Engineer Associate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cyber Security Analyst,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Business Analyst 4 - Lansing, MI - 12+ Months ...",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Systems Administrator,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# 1. Separate out skill columns (everything except job_title)
skill_cols = df_tech_bin.columns.drop("job_title")

# 2. Compute counts (number of 1s) per skill
skill_counts = df_tech_bin[skill_cols].sum(axis=0)



# 3. Keep only skills with count > 5
# cols_to_keep = skill_counts[(skill_counts > 1) & (skill_counts <= 5)].index
cols_to_keep = skill_counts[skill_counts >1].index
# cols_to_keep = skill_counts.index #not removing any skill


# 4. Rebuild the dataframe: job_title + filtered skills
df_tech_bin_filtered = pd.concat(
    [df_tech_bin[["job_title"]], df_tech_bin[cols_to_keep]],
    axis=1
)

print("Original skill columns:", len(skill_cols))
print("Kept skill columns:", len(cols_to_keep))
df_tech_bin_filtered.head()


Original skill columns: 6397
Kept skill columns: 2349


,job_title,.net,.net maui,3d modeling,ability to work in a fastpaced environment,ability to work independently,ability to work independently and as part of a team,ability to work under pressure,acceptance criteria,access control,...,writing,wsus,x86 architecture,xaml,xml,xp,xray imaging,yaml,zabbix,zero trust
0,RF Systems Engineer,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Security Engineer Associate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Cyber Security Analyst,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Business Analyst 4 - Lansing, MI - 12+ Months ...",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Systems Administrator,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_tech_bin_filtered.columns


Index(['job_title', '.net', '.net maui', '3d modeling',
       'ability to work in a fastpaced environment',
       'ability to work independently',
       'ability to work independently and as part of a team',
       'ability to work under pressure', 'acceptance criteria',
       'access control',
       ...
       'writing', 'wsus', 'x86 architecture', 'xaml', 'xml', 'xp',
       'xray imaging', 'yaml', 'zabbix', 'zero trust'],
      dtype='object', length=2350)

In [16]:
duplicate_rows = df_tech_bin_filtered[df_tech_bin_filtered.duplicated()]
print("count of duplicates: ",df_tech_bin_filtered.duplicated().sum() )
# print(duplicate_rows)

count of duplicates:  9


In [17]:
df_tech_bin_filtered=df_tech_bin_filtered.drop_duplicates().reset_index(drop=True)
df_tech_bin_filtered = df_tech_bin_filtered[df_tech_bin_filtered['job_title'] != 'Experienced Associate, Software Engineer (Python/SQL)'].reset_index(drop=True)


In [18]:
skills_df = df_tech_bin_filtered.drop(columns=["job_title", "irrelevant_skill","irrelevant_skills"])
skills_job_df=df_tech_bin_filtered

In [19]:
print(skills_df.head()) #only skills
print(skills_df.columns)
print(skills_df.shape)

   .net  .net maui  3d modeling  ability to work in a fastpaced environment  \
0     1          0            0                                           0   
1     0          0            0                                           0   
2     1          0            0                                           0   
3     1          0            0                                           0   
4     0          0            0                                           0   

   ability to work independently  \
0                              0   
1                              0   
2                              0   
3                              0   
4                              0   

   ability to work independently and as part of a team  \
0                                                  0     
1                                                  0     
2                                                  0     
3                                                  0     
4                 

In [20]:
s1=set(skills_df.columns)
s2=set(skills_job_df.columns)
print("only difference between skills df and skills job df (columns):", s2-s1)

only difference between skills df and skills job df (columns): {'irrelevant_skill', 'job_title', 'irrelevant_skills'}


In [21]:
skills_df.to_csv("../data/skills_only_data.csv", index=False)
skills_job_df.to_csv("../data/skills_with_jobtitle_data.csv", index=False)